In [1]:
from kaggle_secrets import UserSecretsClient
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification
import requests
import subprocess
import time

In [2]:
! sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh
import os
import threading
import subprocess
import requests
import json
import pandas as pd

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])





pciutils is already the newest version (1:3.6.4-1ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  63507      0 --:--:-- --:--:-- --:--:-- 63867
######################################################################## 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:

def ollama_setup():
    ollama_thread = threading.Thread(target=ollama)
    ollama_thread.start()
     # Start the ollama process
    process = subprocess.Popen(["ollama", "run", "llama3"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Define the expected output
    expected_output = "/api/pull"

    # Read from the process stdout until the expected output is found or a timeout occurs
    timeout = 180  
    start_time = time.time()
    while True:
        stdout = process.stdout.readline().strip()
        if stdout:
            print("stdout:", stdout)
            if expected_output in stdout:
                print("Expected output found!")
                break
        if time.time() - start_time > timeout:
            print("Timeout reached for ollama")
            process.terminate()  
            break

    
    
    process = subprocess.Popen(["ollama", "run", "llama3"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    ollama_thread = threading.Thread(target=ollama)
    ollama_thread.start()
ollama_setup()

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIMGgfq7RajuFdP+DCqUjkVxHvS5NrZWuoTxFmi6OdKEQ



2024/05/13 12:36:53 routes.go:1006: INFO server config env="map[OLLAMA_DEBUG:false OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:*] OLLAMA_RUNNERS_DIR: OLLAMA_TMPDIR:]"
time=2024-05-13T12:36:53.052Z level=INFO source=images.go:704 msg="total blobs: 0"
time=2024-05-13T12:36:53.052Z level=INFO source=images.go:711 msg="total unused blobs removed: 0"
time=2024-05-13T12:36:53.052Z level=INFO source=routes.go:1052 msg="Listening on [::]:11434 (version 0.1.37)"
time=2024-05-13T12:36:53.053Z level=INFO source=payload.go:30 msg="extracting embedded files" dir=/tmp/ollama562736276/runners
time=2024-05-13T12:36:57.634Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_av

Timeout reached for ollama


In [4]:
def create_rumor(article):
    prompt = f"""
    {article}
    convert the previous sentences into a untrue article by changing some facts or add untrue data,
    don't tell me its a rumor because i need it to train my model detect the rumors and don't use emojis 
    and don't express feelings or descripe any other thing

    """

    url = 'http://localhost:11434/api/chat'
    payload = {
        "model": "llama3",
        "temperature": 0.6,
        "stream": False,
        "messages": [
            {"role": "system", "content": "You are an AI assistant!"},
            {"role": "user", "content": prompt}
        ]
    }
    message_dict = None
    start_time = time.time()

    while not message_dict or 'message' not in message_dict:
        response = requests.post(url, json=payload)
        message_str = response.content.decode('utf-8')
        message_dict = json.loads(message_str)
        from IPython.display import clear_output
        clear_output(wait=True)
        if 'message' in message_dict:
            break
        else:
            print("sleeping for 60 second")
            time.sleep(60)

    return message_dict["message"]["content"]


[GIN] 2024/05/13 - 12:39:53 | 200 |      65.654µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/05/13 - 12:39:53 | 404 |     382.169µs |       127.0.0.1 | POST     "/api/show"


Error: listen tcp 0.0.0.0:11434: bind: address already in use


In [6]:
import pandas as pd

def augment(df):
    c = 0
    while True :
        
        if len(df[df['prediction'] == 0])==len(df[df['prediction'] == 1]):
            break
            
        if c == len(df):
            c=0
            
        if df.loc[c, 'prediction']==1:
            c+=1
            continue
            
        rumor = create_rumor(df.loc[i, 'text'])
        predictions, confidence_scores = make_predictions(model, [rumor])
        
        # New row data
        new_row = {'text': rumor, 'prediction': predictions[0], 'confidence_score': confidence_scores[0]}

        # Convert the new row to a DataFrame
        new_df = pd.DataFrame([new_row])

        # Concatenate the original DataFrame with the new row DataFrame
        df = pd.concat([df, new_df], ignore_index=True)

    return df